In [1]:
import azureml
import json

from azureml.core.model import Model, InferenceConfig
from azureml.core import Workspace, Datastore, Experiment, Environment, ScriptRunConfig, Run 
from azureml.core.dataset import Dataset
from azureml.core.compute import ComputeTarget
from azureml.core.webservice import AciWebservice
from azureml.core.conda_dependencies import CondaDependencies 

In [2]:
ws = Workspace(workspace_name="Cybertron",resource_group="Cybertron-RG",subscription_id="431dbae5-40ca-438a-8daf-77d7d5580b41")
print("Workspace:",ws.name)

# Connect to compute for training
compute_target = ComputeTarget(workspace=ws, name="OptimusPrime")
print("Compute Target:",compute_target.name)

# Connect to the datastore for the training images
ds = Datastore.get_default(ws)
print("Datastore:",ds.name)

# Connect to the experiment
exp = Experiment(workspace=ws, name='Simpsons-PyTorch')
print("Experiment:",exp.name)

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code CTWEV24E8 to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...


Failed to authenticate to tenant '3394946f-5910-4ddc-9764-87f3e7841ec3' due to error 'Get Token request returned http error: 400 and server response: {"error":"interaction_required","error_description":"AADSTS50076: Due to a configuration change made by your administrator, or because you moved to a new location, you must use multi-factor authentication to access '797f4846-ba00-4fd7-ba43-dac1f8f63013'.\r\nTrace ID: b0da616b-e2c6-4c9f-83f5-6311f7d3ca00\r\nCorrelation ID: c8fa2cb3-808e-489c-a3ce-b60023f1a1f2\r\nTimestamp: 2021-11-27 10:00:46Z","error_codes":[50076],"timestamp":"2021-11-27 10:00:46Z","trace_id":"b0da616b-e2c6-4c9f-83f5-6311f7d3ca00","correlation_id":"c8fa2cb3-808e-489c-a3ce-b60023f1a1f2","error_uri":"https://login.microsoftonline.com/error?code=50076","suberror":"basic_action"}'.Will continue to look for other tenants to find subscriptions to which you have access
Failed to authenticate to tenant 'b4c9f32e-da17-4ded-9c95-ce9da38f25d9' due to error 'Get Token request return

Interactive authentication successfully completed.
Workspace: Cybertron
Compute Target: OptimusPrime
Datastore: workspaceblobstore
Experiment: Simpsons-PyTorch


## Deploy PyTorch model

In [7]:
myenv = Environment(name="simpsons-inference")

conda_dep = CondaDependencies()

# You must list azureml-defaults as a pip dependency
conda_dep.add_pip_package("azureml-defaults")
conda_dep.add_pip_package("torch")
conda_dep.add_pip_package("torchvision")
conda_dep.add_pip_package("pillow==5.4.1")

# Adds dependencies to PythonSection of myenv
myenv.python.conda_dependencies=conda_dep

model = Model(ws, name='simpsons-classification-pytorch')
print(model.version)

inference_config = InferenceConfig(
    entry_script = "./pytorch/score.py", 
    environment = myenv
)

deploy_config = AciWebservice.deploy_configuration(
    cpu_cores = 2, 
    memory_gb = 4,
    description='Simpson Lego Classifier'
)

aci_service = Model.deploy(ws, 
    name="simpsons-classification-pt-v1", 
    models = [model], 
    inference_config = inference_config, 
    deployment_config = deploy_config, 
    overwrite = True
)

aci_service.wait_for_deployment(show_output=True)

3
Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-11-11 17:28:59+01:00 Creating Container Registry if not exists.
2021-11-11 17:29:00+01:00 Registering the environment.
2021-11-11 17:29:01+01:00 Use the existing image.
2021-11-11 17:29:01+01:00 Generating deployment configuration.
2021-11-11 17:29:02+01:00 Submitting deployment to compute.
2021-11-11 17:29:07+01:00 Checking the status of deployment simpsons-classification-pt-v1..
2021-11-11 17:30:53+01:00 Checking the status of inference endpoint simpsons-classification-pt-v1.
Failed


Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: e443682b-4abd-4d08-97c2-609b81101c66
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.
	1. Please check the logs for your container instance: simpsons-classification-pt-v1. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs.
	2. You can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.
	3. You can also try to run image cybertron48878463.azurecr.io/azureml/azureml_1566ddc49d4bc958af64b0982f94332f locally. Please refer to https://aka.ms/debugimage#service-launch-fai

WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: e443682b-4abd-4d08-97c2-609b81101c66
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Your container application crashed. This may be caused by errors in your scoring file's init() function.
	1. Please check the logs for your container instance: simpsons-classification-pt-v1. From the AML SDK, you can run print(service.get_logs()) if you have service object to fetch the logs.
	2. You can interactively debug your scoring file locally. Please refer to https://docs.microsoft.com/azure/machine-learning/how-to-debug-visual-studio-code#debug-and-troubleshoot-deployments for more information.
	3. You can also try to run image cybertron48878463.azurecr.io/azureml/azureml_1566ddc49d4bc958af64b0982f94332f locally. Please r

In [55]:
img_name = "https://raw.githubusercontent.com/hnky/dataset-lego-figures/master/_test/Lisa.jpg"
result = aci_service.run(input_data=json.dumps({ "image": img_name}))

print(result)

{'time': '13.74ms', 'prediction': 'Lisa-Simpson', 'confidence': '97.97%'}


## Deploy ONNX model

In [52]:
myenv = Environment(name="simpsons-inference-onnx")
conda_dep = CondaDependencies()

# You must list azureml-defaults as a pip dependency
conda_dep.add_pip_package("azureml-defaults")
conda_dep.add_pip_package("onnxruntime")
conda_dep.add_pip_package("pillow==5.4.1")

# Adds dependencies to PythonSection of myenv
myenv.python.conda_dependencies=conda_dep

model = Model(ws, name='simpsons-classification-onnx')

inference_config = InferenceConfig(
    entry_script = "./onnx/score.py", 
    environment = myenv
)

deploy_config = AciWebservice.deploy_configuration(
    cpu_cores = 2, 
    memory_gb = 4,
    description='Simpson Lego Classifier'
)

In [53]:
aci_service = Model.deploy(ws, 
    name="simpsons-classification-onnx", 
    models = [model], 
    inference_config = inference_config, 
    deployment_config = deploy_config, 
    overwrite = True
)

aci_service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-11-11 16:54:35+01:00 Creating Container Registry if not exists.
2021-11-11 16:54:35+01:00 Registering the environment.
2021-11-11 16:54:36+01:00 Use the existing image.
2021-11-11 16:54:36+01:00 Generating deployment configuration.
2021-11-11 16:54:37+01:00 Submitting deployment to compute.
2021-11-11 16:54:40+01:00 Checking the status of deployment simpsons-classification-onnx..
2021-11-11 16:55:07+01:00 Checking the status of inference endpoint simpsons-classification-onnx.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [54]:
img_name = "https://raw.githubusercontent.com/hnky/dataset-lego-figures/master/_test/Lisa.jpg"
result = aci_service.run(input_data=json.dumps({ "image": img_name}))

print(result)

{'time': '29.12ms', 'prediction': 'Lisa-Simpson', 'confidence': '97.97%'}


In [51]:
aci_service.get_logs()

In [40]:
aci_service.get_logs()

'2021-11-11T15:28:43,260644044+00:00 - nginx/run \n2021-11-11T15:28:43,261212247+00:00 - rsyslog/run \n2021-11-11T15:28:43,261708550+00:00 - iot-server/run \n2021-11-11T15:28:43,272072405+00:00 - gunicorn/run \nDynamic Python package installation is disabled.\nStarting HTTP server\nEdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...\n2021-11-11T15:28:43,343675089+00:00 - iot-server/finish 1 0\n2021-11-11T15:28:43,345021396+00:00 - Exit code 1 is normal. Not restarting iot-server.\nStarting gunicorn 20.1.0\nListening at: http://127.0.0.1:31311 (11)\nUsing worker: sync\nworker timeout is set to 300\nBooting worker with pid: 40\nSPARK_HOME not set. Skipping PySpark Initialization.\nInitializing logger\n2021-11-11 15:28:43,970 | root | INFO | Starting up app insights client\nlogging socket was found. logging is available.\nlogging socket was found. logging is available.\n2021-11-11 15:28:43,970 | root | INFO | Starting up request id generator\n2021-11-11 15:28:43,970